In [15]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import warnings
import os

warnings.filterwarnings('ignore')

# Add src to path
import sys
sys.path.append('..')

from src.config import (
    SEQUENCES_DIR, BEST_MODEL_PATH, CHECKPOINTS_DIR, TRAINING_FIGURES_DIR,
    LOGS_DIR, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN,
    ENCODER_HIDDEN_SIZE, ENCODER_NUM_LAYERS, ENCODER_DROPOUT, ENCODER_BIDIRECTIONAL,
    DECODER_HIDDEN_SIZE, DECODER_NUM_LAYERS, DECODER_DROPOUT,
    BATCH_SIZE, LEARNING_RATE, WEIGHT_DECAY, NUM_EPOCHS,
    EARLY_STOPPING_PATIENCE, GRADIENT_CLIP, TEACHER_FORCING_RATIO,
    LR_SCHEDULER, DEVICE, RANDOM_SEED, set_seed
)
from src.dataset import create_dataloaders
from src.model import build_model
from src.train import train
from src.utils import print_gpu_info, save_figure, load_json

# Set random seed
set_seed(RANDOM_SEED)

# Create logs directory
os.makedirs(LOGS_DIR, exist_ok=True)

print("Libraries imported successfully!")
print_gpu_info()

Libraries imported successfully!
GPU: NVIDIA GeForce RTX 3060
CUDA Version: 11.8
Memory Allocated: 24.25 MB
Memory Cached: 172.00 MB


## 6.1 Load Sequences

In [16]:
# Load sequences
X_train = np.load(os.path.join(SEQUENCES_DIR, 'X_train.npy'))
y_train = np.load(os.path.join(SEQUENCES_DIR, 'y_train.npy'))
X_val = np.load(os.path.join(SEQUENCES_DIR, 'X_val.npy'))
y_val = np.load(os.path.join(SEQUENCES_DIR, 'y_val.npy'))
X_test = np.load(os.path.join(SEQUENCES_DIR, 'X_test.npy'))
y_test = np.load(os.path.join(SEQUENCES_DIR, 'y_test.npy'))

# Load metadata
metadata = load_json(os.path.join(SEQUENCES_DIR, 'metadata.json'))

print("Data loaded:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

Data loaded:
X_train: (19394, 24, 22)
y_train: (19394, 5)
X_val: (4156, 24, 22)
y_val: (4156, 5)
X_test: (4157, 24, 22)
y_test: (4157, 5)


In [17]:
# Get dimensions
n_features = X_train.shape[2]
print(f"\nNumber of input features: {n_features}")
print(f"Input sequence length: {INPUT_SEQ_LEN}")
print(f"Output sequence length: {OUTPUT_SEQ_LEN}")


Number of input features: 22
Input sequence length: 24
Output sequence length: 5


## 6.2 Create DataLoaders

In [18]:
# Create DataLoaders
train_loader, val_loader, test_loader = create_dataloaders(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test,
    batch_size=BATCH_SIZE,
    num_workers=0
)

print(f"\nTrain batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

Created DataLoaders with batch_size=64

Train batches: 304
Val batches: 65
Test batches: 65


## 6.3 Build Model

In [19]:
# Build Encoder-Decoder model
model = build_model(
    input_size=n_features,
    hidden_size=ENCODER_HIDDEN_SIZE,
    num_layers=ENCODER_NUM_LAYERS,
    dropout=ENCODER_DROPOUT,
    bidirectional=ENCODER_BIDIRECTIONAL,
    output_seq_len=OUTPUT_SEQ_LEN,
    device=DEVICE
)

Model built on cuda
Total parameters: 816,001
Trainable parameters: 816,001


In [20]:
# Model architecture summary
print("\nModel Architecture:")
print("=" * 60)
print(model)


Model Architecture:
Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(22, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
    (fc_hidden): Linear(in_features=256, out_features=128, bias=True)
    (fc_cell): Linear(in_features=256, out_features=128, bias=True)
  )
  (decoder): Decoder(
    (lstm): LSTM(1, 128, num_layers=2, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=128, out_features=1, bias=True)
  )
)


## 6.4 Training Configuration

In [21]:
# Print training configuration
print("Training Configuration:")
print("=" * 60)
print(f"Device: {DEVICE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Weight decay: {WEIGHT_DECAY}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Early stopping patience: {EARLY_STOPPING_PATIENCE}")
print(f"Gradient clipping: {GRADIENT_CLIP}")
print(f"Teacher forcing ratio: {TEACHER_FORCING_RATIO}")
print(f"LR scheduler: {LR_SCHEDULER}")
print("=" * 60)

Training Configuration:
Device: cuda
Batch size: 64
Learning rate: 0.001
Weight decay: 1e-05
Epochs: 100
Early stopping patience: 15
Gradient clipping: 1.0
Teacher forcing ratio: 0.5
LR scheduler: ReduceLROnPlateau


## 6.4a Load Optuna Best Params (Optional)

Run this cell if you have already run Optuna optimization and want to use the best parameters.

In [22]:
# Load best parameters from Optuna (if available)
best_params_path = os.path.join(LOGS_DIR, 'best_params.json')

USE_OPTUNA_PARAMS = True  # Set to True to use Optuna parameters

if USE_OPTUNA_PARAMS and os.path.exists(best_params_path):
    optuna_params = load_json(best_params_path)
    print("Loaded Optuna best parameters:")
    for k, v in optuna_params.items():
        print(f"  {k}: {v}")
    
    # Override config values
    ENCODER_HIDDEN_SIZE = optuna_params.get('hidden_size', ENCODER_HIDDEN_SIZE)
    ENCODER_NUM_LAYERS = optuna_params.get('num_layers', ENCODER_NUM_LAYERS)
    ENCODER_DROPOUT = optuna_params.get('dropout', ENCODER_DROPOUT)
    LEARNING_RATE = optuna_params.get('learning_rate', LEARNING_RATE)
    BATCH_SIZE = optuna_params.get('batch_size', BATCH_SIZE)
    WEIGHT_DECAY = optuna_params.get('weight_decay', WEIGHT_DECAY)
    TEACHER_FORCING_RATIO = optuna_params.get('teacher_forcing_ratio', TEACHER_FORCING_RATIO)
    
    print("\nConfig values updated!")
else:
    print("Using default config parameters")

Loaded Optuna best parameters:
  hidden_size: 64
  num_layers: 2
  dropout: 0.24822872742761123
  learning_rate: 0.0015517954134162649
  batch_size: 64
  weight_decay: 0.00037944994831423065
  teacher_forcing_ratio: 0.648343601345059

Config values updated!


## 6.5 Train Model

In [23]:
# Train model with logging
history = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    teacher_forcing_ratio=TEACHER_FORCING_RATIO,
    gradient_clip=GRADIENT_CLIP,
    early_stopping_patience=EARLY_STOPPING_PATIENCE,
    lr_scheduler_type=LR_SCHEDULER,
    checkpoint_dir=CHECKPOINTS_DIR,
    best_model_path=BEST_MODEL_PATH,
    device=DEVICE,
    log_dir=LOGS_DIR  # Enable logging
)

2025-12-05 04:03:08 | INFO     | src.logger:setup_logger:67 | Log file: d:\DeepLearning_final\logs\training.log
2025-12-05 04:03:08 | INFO     | src.train:train:134 | Training on cuda
2025-12-05 04:03:08 | INFO     | src.logger:log_hyperparams:140 | ============================================================
2025-12-05 04:03:08 | INFO     | src.logger:log_hyperparams:141 | Hyperparameters:
2025-12-05 04:03:08 | INFO     | src.logger:log_hyperparams:143 |   num_epochs: 100
2025-12-05 04:03:08 | INFO     | src.logger:log_hyperparams:143 |   learning_rate: 0.0015517954134162649
2025-12-05 04:03:08 | INFO     | src.logger:log_hyperparams:143 |   weight_decay: 0.00037944994831423065
2025-12-05 04:03:08 | INFO     | src.logger:log_hyperparams:143 |   teacher_forcing_ratio: 0.648343601345059
2025-12-05 04:03:08 | INFO     | src.logger:log_hyperparams:143 |   gradient_clip: 1.0
2025-12-05 04:03:08 | INFO     | src.logger:log_hyperparams:143 |   early_stopping_patience: 15
2025-12-05 04:03:08 

2025-12-05 04:03:11 | SUCCESS  | src.logger:log_best_model:123 | New best model at epoch 1 with val_loss=0.006403
2025-12-05 04:03:11 | INFO     | src.logger:log_best_model:124 | Model saved to: d:\DeepLearning_final\models\best_model.pth
2025-12-05 04:03:11 | INFO     | src.logger:log_epoch:113 | Epoch   1 | Train Loss: 0.014411 | Val Loss: 0.006403 | LR: 1.55e-03 | Time: 2.7s


2025-12-05 04:03:14 | INFO     | src.logger:log_epoch:113 | Epoch   2 | Train Loss: 0.003758 | Val Loss: 0.007568 | LR: 1.55e-03 | Time: 2.4s


2025-12-05 04:03:16 | SUCCESS  | src.logger:log_best_model:123 | New best model at epoch 3 with val_loss=0.003764
2025-12-05 04:03:16 | INFO     | src.logger:log_best_model:124 | Model saved to: d:\DeepLearning_final\models\best_model.pth
2025-12-05 04:03:16 | INFO     | src.logger:log_epoch:113 | Epoch   3 | Train Loss: 0.003030 | Val Loss: 0.003764 | LR: 1.55e-03 | Time: 2.5s


2025-12-05 04:03:19 | INFO     | src.logger:log_epoch:113 | Epoch   4 | Train Loss: 0.002648 | Val Loss: 0.004449 | LR: 1.55e-03 | Time: 2.5s


...

2025-12-05 04:04:27 | INFO     | src.logger:log_epoch:113 | Epoch  32 | Train Loss: 0.001092 | Val Loss: 0.003220 | LR: 7.76e-04 | Time: 2.4s


2025-12-05 04:04:30 | INFO     | src.logger:log_epoch:113 | Epoch  33 | Train Loss: 0.001046 | Val Loss: 0.003149 | LR: 7.76e-04 | Time: 3.0s


2025-12-05 04:04:32 | INFO     | src.logger:log_epoch:113 | Epoch  34 | Train Loss: 0.000952 | Val Loss: 0.003244 | LR: 3.88e-04 | Time: 2.6s


2025-12-05 04:04:35 | INFO     | src.logger:log_epoch:113 | Epoch  35 | Train Loss: 0.000892 | Val Loss: 0.003276 | LR: 3.88e-04 | Time: 2.4s


2025-12-05 04:04:37 | INFO     | src.logger:log_epoch:113 | Epoch  36 | Train Loss: 0.000917 | Val Loss: 0.003115 | LR: 3.88e-04 | Time: 2.4s
2025-12-05 04:04:37 | WARNING  | src.logger:log_early_stopping:128 | Early stopping triggered at epoch 36 (patience=15)
2025-12-05 04:04:37 | SUCCESS  | src.logger:log_training_complete:132 | Training completed in 1.5 minutes | Best epoch: 21 | Best val_loss: 0.002864
2025-12-05 04:04:37 | INFO     | src.train:train:250 | Loaded best model from epoch 21


## 6.6 Training Visualization

In [27]:
# Plot learning curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
epochs = range(1, len(history['train_loss']) + 1)
axes[0].plot(epochs, history['train_loss'], 'b-', label='Train Loss')
axes[0].plot(epochs, history['val_loss'], 'r-', label='Validation Loss')
axes[0].axvline(x=history['best_epoch'] + 1, color='green', linestyle='--', label=f'Best Epoch ({history["best_epoch"]+1})')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True)

# Learning rate
axes[1].plot(epochs, history['learning_rate'], 'g-')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Learning Rate')
axes[1].set_title('Learning Rate Schedule')
axes[1].set_yscale('log')
axes[1].grid(True)

plt.tight_layout()
os.makedirs(TRAINING_FIGURES_DIR, exist_ok=True)
save_figure(fig, os.path.join(TRAINING_FIGURES_DIR, 'learning_curves.png'))
plt.show()

Saved: d:\DeepLearning_final\results\figures\training\learning_curves.png


In [28]:
# Training summary
print("\n" + "=" * 60)
print("TRAINING SUMMARY")
print("=" * 60)
print(f"Total epochs trained: {len(history['train_loss'])}")
print(f"Best epoch: {history['best_epoch'] + 1}")
print(f"Best validation loss: {history['best_val_loss']:.6f}")
print(f"Final training loss: {history['train_loss'][-1]:.6f}")
print(f"Final validation loss: {history['val_loss'][-1]:.6f}")
print(f"Model saved to: {BEST_MODEL_PATH}")
print("=" * 60)


TRAINING SUMMARY
Total epochs trained: 36
Best epoch: 21
Best validation loss: 0.002864
Final training loss: 0.000917
Final validation loss: 0.003115
Model saved to: d:\DeepLearning_final\models\best_model.pth


## 6.7 Save Training History

In [29]:
# Save training history
from src.utils import save_json

history_to_save = {
    'train_loss': [float(x) for x in history['train_loss']],
    'val_loss': [float(x) for x in history['val_loss']],
    'learning_rate': [float(x) for x in history['learning_rate']],
    'best_epoch': int(history['best_epoch']),
    'best_val_loss': float(history['best_val_loss'])
}

save_json(history_to_save, os.path.join(TRAINING_FIGURES_DIR, 'training_history.json'))
print("Training history saved!")

Saved: d:\DeepLearning_final\results\figures\training\training_history.json
Training history saved!


## Summary

**Model Training completed:**
1. ✅ Loaded sequence data
2. ✅ Created DataLoaders
3. ✅ Built Encoder-Decoder model
4. ✅ Trained with GPU
5. ✅ Early stopping applied
6. ✅ Saved best model
7. ✅ Visualized learning curves

**Next step:** Evaluation (07_Evaluation.ipynb)